In [5]:
import os
from rosemary import jpt_setup; jpt_setup()

from transformers.trainer_utils import get_last_checkpoint

from llm.submit import (
    multiline_to_singleline,
    submit_job_ccc,
    submit_job_aimos,
    submit_job,
    get_run_statistics)
import pandas as pd
import json
import platform
import tempfile
import subprocess
import shlex
import datetime
import itertools

arch = platform.uname().processor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Finetuning with openinstruct/finetune.py


In [8]:
# job_name = 'ft'
# test_run = 1
# test_run = bool(test_run)

# queue = 'x86_12h' # 'x86_12h'
# num_cpus = 20
# num_gpus = 1
# cpu_mem = 32
# require = 'a100_80gb'

# # model_name_or_path = 'mosaicml/mpt-7b'; max_seq_length = 2048
# # model_name_or_path = 'gpt2'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-Large'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-xl'; max_seq_length = 1024
# model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048


# train_file = 'data/processed/oasst1/oasst1_data.jsonl'; train_file_short = 'oasst1'
# train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
# # train_file = 'data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl'; train_file_short = 'human_mix_shuffled'

# output_dir = f"results/{model_name_or_path.replace('/', ':')}_{train_file_short}"
# if test_run:
#     output_dir = 'jpt_' + output_dir

# use_deepspeed = False
# # deepspeed_config_file = 'ds_configs/stage3_no_offloading_accelerate_setauto.conf'
# # deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate.conf'
# deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate_setauto.conf'

# use_lora = True
# lora_rank = 4
# lora_alpha = lora_rank
# lora_dropout = 0.05

# batch_size_per_gpu = 1
# total_batch_size = 128
# mixed_precision = 'bf16' # 'bf16', 'fp16'
# checkpointing_steps = None # every n steps, where n='1' or every 'epoch'

# gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

# print(f"Training {model_name_or_path} "
#       f"using {num_gpus} GPUs, "
#       f"{batch_size_per_gpu} batch size per GPU, "
#       f"{gradient_acc_steps} gradient accumulation steps.")

# # do use fast tokenizer since mpt-7b does not have a fast tokenizer counter-part
# #     --use_slow_tokenizer \
# # do not use flash attention, since having problem installing flash-attn with cuda 12.1
# #     --use_flash_attn \

# cmd = f"""
# {'!cd .. && ' if test_run else ''}accelerate launch \
#     --mixed_precision {mixed_precision} \
#     --num_machines 1 \
#     --num_processes {num_gpus} \
#     {'--use_deepspeed' if use_deepspeed else ''}
#     {'--deepspeed_config_file '+deepspeed_config_file if use_deepspeed else ''}
#     open_instruct/finetune.py \
#     --model_name_or_path {model_name_or_path} \
#     --tokenizer_name {model_name_or_path} \
#     --train_file {train_file} \
#     --max_seq_length {max_seq_length} \
#     {'--use_lora' if use_lora else ''}
#     --lora_rank {lora_rank} \
#     --lora_alpha {lora_alpha} \
#     --lora_dropout {lora_dropout} \
#     --preprocessing_num_workers 16 \
#     --per_device_train_batch_size {batch_size_per_gpu} \
#     --gradient_accumulation_steps {gradient_acc_steps} \
#     --learning_rate 2e-5 \
#     --lr_scheduler_type linear \
#     --warmup_ratio 0.03 \
#     --weight_decay 0. \
#     --num_train_epochs 2 \
#     --output_dir {output_dir} \
#     --with_tracking \
#     --report_to tensorboard \
#     {'--checkpointing_steps '+str(checkpointing_steps) if checkpointing_steps else ''}
#     --logging_steps 1
# """

# # things to test to see its effects on (1) eval perf (2) runtime.
# #
# # - int8
# # - mixed_precision bf16 or no
# # - with/without LoRA
# # - LoRA's rank/alpha (alpha typically set to 2*rank)
# # - batch size
# # - micro-batch size (largest without running out of memory)


# cmd = multiline_to_singleline(cmd)
# if test_run:
#     print()
#     print(cmd)

# shell_scripts = shell_scripts_template.format(
#     cmd=cmd,
#     log_dir=os.getcwd(),
#     save_dir=output_dir)
# out = submit_job_ccc(
#     shell_scripts, 
#     job_name=job_name, 
#     queue=queue,
#     num_cpus=num_cpus,
#     cpu_mem=cpu_mem,
#     require=require,
#     num_gpus=num_gpus,
#     test_run=test_run,
# )
# if not test_run:
#     print(out)

In [9]:
# llama7b+lora, micro-bsz=1, bsz=128, 
#     66gb gpu mem, in it 28gb for torch tensor. so fit on 1 a100_80gb

#   1%|▏         | 57/4222 [22:24<25:53:47, 22.38s/it]07/11/2023 23:13:55 - INFO - 
# __main__ -   Step: 57, LR: 9.047619047619049e-06, Loss: 2.2924644947052
# 


# !cd .. && accelerate launch --mixed_precision bf16 --num_machines 1 --num_processes 1 open_instruct/finetune.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --use_lora --lora_rank 4 --lora_alpha 4 --lora_dropout 0.05 --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 128 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --num_train_epochs 2 --output_dir results/huggyllama:llama-7b_human_mixjpt_results/huggyllama:llama-7b_human_mix --with_tracking --report_to tensorboard --logging_steps 1





# Finetuning with openinstruct/finetune_trainer.py


In [8]:
# runtime:
# shared: humanmix, max_sequence_length=2048.
# just a single process if num_gpus=1 otherwise use python fsdp with gradient-checkpointing.
#
# llama7b, micro-bsz=2, grad-ckpt, 2xa100_80gb: 27s/it, 32hrs
# 
# llama7b+lora(r=4),   micro-bsz=1, no-grad-ckpt,a100_80gb: 27s/it, 32hrs, 66gb gpu mem
# llama7b+lora(r=16),  micro-bsz=1, no-grad-ckpt,a100_80gb: 24s/it, 28hrs, 67gb gpu mem
# llama7b+lora(r=256), micro-bsz=1, no-grad-ckpt,a100_80gb: 24s/it, 28hrs, 70gb gpu mem
#
# llama7b+lora(r=4),   micro-bsz=1,  grad-ckpt,  a100_80gb: 32s/it, 38hrs
# llama7b+lora(r=256), micro-bsz=1,  grad-ckpt,  a100_80gb, 37s/it, 43hrs# llama7b+lora(r=4),   micro-bsz=2,  grad-ckpt,  a100_80gb: 32s/it, 38hrs (+20% runtime with grad-ckpt)
# llama7b+lora(r=4),   micro-bsz=16, grad-ckpt,  a100_80gb: 68s/it, 80hrs, 75gb gpu mem
# llama7b+lora(r=128), micro-bsz=16, grad-ckpt,  a100_80gb: 70s/it, 82hrs
# 
# take-aways
# - larger batch should speed up training. however, it also implies that the batch 
#   will be padded to longer sequence length, due to chat data, thereby increasing
#   compute required to process a batch. Therefore, micro-bsz=1 seems to be ok.
#

# aimos
# shared: humanmix, max_sequence_length=2048. 1 node = 6x v100_32gb

# take-aways: 
#   (1) fsdp (v4.28.1, v4.32.0.dev0 are pretty similar in terms of speed. don't use v4.31.0)
#   (2) micro-bsz=1->2 seems to be the best here. leads to ~30% reduction in runtime.
#   (3) increasing #nodes almost linear reduction in time, e.g., Use 4x nodes cost 30% time (25% if linear.)

# llama7b, micro-bsz=1, fsdp, grad-ckpt, float16, mp=none, nodes=1, 74s/it, 89hrs, loss=0
# llama7b, micro-bsz=1, fsdp, grad-ckpt, float16, mp=fp16, nodes=1, 72s/it, 86hrs
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float16, mp=none, nodes=1, 52s/it, 66hrs, loss=0
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float16, mp=fp16, nodes=1, 49s/it, 61hrs
# - resume from lastest checkpt (trained 4.28.1, resume 4.32.0.dev0), did
#   did not found `optimizer.bin` 
# llama7b, micro-bsz=3, fsdp, grad-ckpt, float16, mp=fp16, nodes=1, oom
# llama7b, micro-bsz=3, fsdp, grad-ckpt, float16, mp=none, nodes=1, oom
# llama7b, micro-bsz=4, fsdp, grad-ckpt, float16, mp=fp16, nodes=1, oom
# llama7b, micro-bsz=4, fsdp, grad-ckpt, float16, mp=none, nodes=1, oom
# llama7b, micro-bsz=1, fsdp, no-grad-ckpt, float16, mp=fp16, nodes=1, oom
# llama7b, micro-bsz=2, fsdp, no-grad-ckpt, float16, mp=fp16, nodes=1, oom
# # nodes > 1
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float16, mp=fp16, nodes=4, 14s/it, 18hrs

# take-away: 
#   (1) torch_dtype=float16 gives loss=0. setting torch_dtype=float32 solves the issue but takes more memory and compute
#   (2) mbsz=2 oom for nodes=1 but works fine with nodes=2. more nodes -> potentially larger mbsz.

# llama7b, micro-bsz=1, fsdp, grad-ckpt, float32, mp=none, nodes=1, oom
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float32, mp=none, nodes=1, oom
# llama7b, micro-bsz=1, fsdp, grad-ckpt, float32, mp=fp16, nodes=1, 139s/it, 166hrs
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float32, mp=fp16, nodes=1, oom
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float32, mp=fp16, nodes=2, 33s/it, 41hrs, loss!=0
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float32, mp=fp16, nodes=4, 17s/it, 21hrs
# llama7b, micro-bsz=3, fsdp, grad-ckpt, float32, mp=fp16, nodes=4, 18s/it, 21hrs
# llama7b, micro-bsz=4, fsdp, grad-ckpt, float32, mp=fp16, nodes=4, oom
# llama7b, micro-bsz=2, fsdp, grad-ckpt, float32, mp=fp16, nodes=5, 4s/it, 21.7hrs


#
# deepspeed
# shared: llama7b, deepspeed, grad-ckpt
# take-aways
#   (1) with deepspeed, using mixed-precision gives x50% speed improvement
#   (2) no loss overflow issues. deepspeed has good mixed-precision integration, loss_scaler handles it.
# 
# llama7b, micro-bsz=1, deepspeed(s=3,offload), float32, mp=fp16, nodes=1, 161s/it, 192hrs, loss ok, loss_scaler early on in use.
# llama7b, micro-bsz=1, deepspeed(s=3,offload), float32, mp=none, nodes=1, 259s/it, 309hrs, loss ok.
# llama7b, micro-bsz=1, deepspeed(s=3,offload), float16, mp=fp16, nodes=1, 169s/it, 202hrs, loss ok. loss_scaler early on in use.
# llama7b, micro-bsz=1, deepspeed(s=3,offload), float16, mp=none, nodes=1, 258s/it, 307hrs, loss ok.
#
# llama7b, micro-bsz=2, deepspeed(s=3,offload), float32, mp=fp16, nodes=1, 108s/it, 135hrs, loss ok.
# llama7b, micro-bsz=4, deepspeed(s=3,offload), float32, mp=fp16, nodes=1,  96s/it, 120hrs, loss ok.
# llama7b, micro-bsz=1, deepspeed(s=3), float32, mp=fp16, nodes=1, 123s/it, 147hrs, loss ok.
# llama7b, micro-bsz=2, deepspeed(s=3), float32, mp=fp16, nodes=1, 66s/it, 83hrs, loss ok.
# llama7b, micro-bsz=4, deepspeed(s=3), float32, mp=fp16, nodes=1, oom
#
# nodes>1
# llama7b, micro-bsz=2, deepspeed(s=3), float32, mp=fp16, nodes=2, error with downloading config.json
# llama7b, micro-bsz=2, deepspeed(s=3), float32, mp=fp16, nodes=4, 
# llama7b, micro-bsz=4, deepspeed(s=3), float32, mp=fp16, nodes=4, 16s/it, 21hrs
# llama7b, micro-bsz=6, deepspeed(s=3), float32, mp=fp16, nodes=4,  3s/it, 17hrs

shell_scripts_template_slurm = """
echo "Running on $SLURM_JOB_NODELIST"
echo "======"

master_addr=$(scontrol show hostnames "$SLURM_JOB_NODELIST" | head -n 1)
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/

echo {cmd}
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$SLURM_JOB_ID*.out" ] && mv {log_dir}/$SLURM_JOB_ID*.out {save_dir}
"""

shell_scripts_template_lsf = """
echo "Running on $LSB_DJOB_HOSTFILE"
echo "======"

master_addr=$(head -n 1 "$LSB_DJOB_HOSTFILE")
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

echo {cmd}
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$LSB_JOBID*.out" ] && mv {log_dir}/$LSB_JOBID*.out {save_dir}
"""

shell_scripts_template = shell_scripts_template_slurm \
    if arch == 'ppc64le' else shell_scripts_template_lsf

print(shell_scripts_template)


echo "Running on $SLURM_JOB_NODELIST"
echo "======"

master_addr=$(scontrol show hostnames "$SLURM_JOB_NODELIST" | head -n 1)
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/

echo {cmd}
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$SLURM_JOB_ID*.out" ] && mv {log_dir}/$SLURM_JOB_ID*.out {save_dir}



In [5]:
t = '5:31:18'
n = 1148
total = 4512; nnodes = 4
# total = 4296; nnodes = 1
# total = 2254; nnodes = 2
# total = 1128; nnodes = 4
# total = 1074; nnodes = 4
# total = 1252; nnodes = 4

l = [int(x) for x in t.split(':')]
t = l[0]*60*60+l[1]*60+l[2]
# t = t/60/60 # in hr

print(f'{t/n/nnodes:.0f}s/it, {t/n*total/60/60:.1f}hrs')

4s/it, 21.7hrs


In [55]:
l = []
p = '../data/processed/'
for x in os.listdir(p):
    y = os.path.join(p, x)
    if os.path.isdir(y):
        d = os.path.join(y, os.listdir(y)[0])
    else:
        continue
    d = d[3:]
    if 'shuffled' in d:
        continue
#     print(f"train_file = '{d}'; abbr_train_file = '{x}'")
    print(x)

cot
sharegpt
dolly
gpt4_alpaca
flan_v2
super_ni
stanford_alpaca
baize
code_alpaca
self_instruct
unnatural_instructions
oasst1


In [85]:
job_name = 'ft'
test_run = 1
test_run = bool(test_run)
# specify `job_duration` to chain jobs for >12hr jobs.
job_duration = 6 if arch == 'ppc64le' else 12
job_duration = 6
# shell_scripts_modification_fn = lambda x: x.replace('--overwrite_output_dir ', '')

nodes = 5 # 128/(5*6*2)~=2.1
num_cpus = 144 if arch == 'ppc64le' else 32
cpu_mem = 512 if arch == 'ppc64le' else 64
num_gpus = 1; gpu_type = 'a100_80gb'
num_gpus = 6; gpu_type = 'v100'
debug_mode = test_run

overwrite_output_dir = True if test_run else False # always continue from ckpt if run from cluster.
save_strategy = 'steps'
save_steps = 100

# model_name_or_path = 'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'
model_name_or_path = 'results/baselines/huggyllama/llama-7b'; abbr_model_name = 'llama-7b'
# model_name_or_path = 'results/baselines/NousResearch/Llama-2-7b-hf'; abbr_model_name = 'llama2-7b'
max_seq_length = 2048
# model_name_or_path = 'mosaicml/mpt-7b'; abbr_model_name = 'mpt-7b'; max_seq_length = 2048
# model_name_or_path = 'gpt2'; abbr_model_name = 'gpt2; max_seq_length = 1024


train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; abbr_train_file = 'humanmix'
# train_file = 'data/processed/dolly_oasst1.jsonl'; abbr_train_file = 'dolly:oasst1'
# train_file = 'data/processed/cot_flanv2.jsonl'; abbr_train_file = 'cot:flanv2'

# train_file = 'data/processed/super_ni/super_ni_data.jsonl'; abbr_train_file = 'super_ni'
# train_file = 'data/processed/cot/cot_data.jsonl'; abbr_train_file = 'cot'
# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2'
# train_file = 'data/processed/dolly/dolly_data.jsonl'; abbr_train_file = 'dolly'
# train_file = 'data/processed/oasst1/oasst1_data.jsonl'; abbr_train_file = 'oasst1'

# train_file = 'data/processed/stanford_alpaca/stanford_alpaca_data.jsonl'; abbr_train_file = 'stanford_alpaca'
# train_file = 'data/processed/baize/baize_data.jsonl'; abbr_train_file = 'baize'
# train_file = 'data/processed/self_instruct/self_instruct_data.jsonl'; abbr_train_file = 'self_instruct'

# train_file = 'data/processed/code_alpaca/code_alpaca_data.jsonl'; abbr_train_file = 'code_alpaca'
# train_file = 'data/processed/unnatural_instructions/unnatural_instructions_data.jsonl'; abbr_train_file = 'unnatural_instructions'
# train_file = 'data/processed/sharegpt/sharegpt_data.jsonl'; abbr_train_file = 'sharegpt'
# train_file = 'data/processed/gpt4_alpaca/gpt4_alpaca_data.jsonl'; abbr_train_file = 'gpt4_alpaca'

job_name += ':'+abbr_train_file


num_train_epochs = 2
batch_size_per_gpu = 2
total_batch_size = 128 # 128
gradient_acc_steps = int(total_batch_size/(num_gpus*nodes)/batch_size_per_gpu)
optimizer = 'adamw_hf' # 'adafactor'

deepspeed = ''; fsdp = False if num_gpus == 1 else "full_shard auto_wrap"  # full_shard, shard_grad_op
if 'gpt2' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'GPT2Block'
elif 'llama' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'LlamaDecoderLayer'
elif 'mpt' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'MPTBlock'
else: raise ValueError('Not sure how to set `fsdp_transformer_layer_cls_to_wrap`')
    
# deepspeed = './ds_configs/ds_zero3_cpu_offload.json'; fsdp = False
# deepspeed = './ds_configs/ds_zero3.json'; fsdp = False
# deepspeed = './ds_configs/stage3_no_offloading.conf'; fsdp = False # error with loading... something wrong with the config.


if fsdp and deepspeed:
    raise ValueError('either fsdp or deepspeed, not both')

use_lora = False
lora_rank = 256 # test {8, 16, 32, 128} # just [128, 8] for now.
lora_alpha = lora_rank 
lora_dropout = 0.05

mixed_precision = 'fp16' if arch == 'ppc64le' else 'bf16' # mixed_precision = ''
torch_dtype = 'bfloat16' if arch=='x86_64' else 'float16'; torch_dtype = 'float32'

gradient_checkpointing = True
load_in_8bit = False

if use_lora:
    abbr_model_name += f'+lora(r={lora_rank},a={lora_alpha})'
output_dirname = f"{abbr_model_name}_{abbr_train_file}"
if test_run:
    output_dirname = 'jpt_'+output_dirname
# if not test_run:
#     output_dirname += \
#         ('_fsdp='+fsdp.split(' ')[0] if fsdp else '')+\
#         ('_deepspeed='+os.path.basename(deepspeed).split('.')[0] if deepspeed else '')+\
#         '_gradckpt='+str(gradient_checkpointing)+\
#         '_mbsz='+str(batch_size_per_gpu)+\
#         '_dtype='+torch_dtype+\
#         ('_mp='+str(mixed_precision) if mixed_precision else '_mp=none')+\
#         '_seqlen='+str(max_seq_length)+\
#         '_nodes='+str(nodes)
output_dir = os.path.join('results', output_dirname)


print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{batch_size_per_gpu} batch size per GPU, "
      f"{gradient_acc_steps} gradient accumulation steps.")


if nodes == 1:
    exe = 'python' if num_gpus==1 else \
        f"torchrun --nproc_per_node={num_gpus} --master_port=10002"
else:
    exe = f"torchrun --nnodes={nodes} --nproc_per_node={num_gpus} --rdzv-id=$SLURM_JOB_ID --rdzv-backend=c10d --rdzv-endpoint=$RDZV_ENDPOINT"

if test_run:
    exe = f"CUDA_VISIBLE_DEVICES={','.join(map(str, range(num_gpus)))} {exe}"
if test_run and debug_mode:
    exe = 'TORCH_CPP_LOG_LEVEL=INFO NCCL_DEBUG=INFO LOGLEVEL=INFO ' + exe
    error_file='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/error_file'
    exe = f'TORCHELASTIC_ERROR_FILE={error_file} {exe}'

#     {'--tf32 True' if arch == 'x86_64' else ''} \

cmd = f"""
{'!cd .. && ' if test_run else ''}{exe}
    open_instruct/finetune_trainer.py \
    --model_name_or_path={model_name_or_path} \
    --tokenizer_name={model_name_or_path} \
    {'--load_in_8bit' if load_in_8bit else ''} \
    --use_fast_tokenizer=True \
    --train_file={train_file} \
    --max_seq_length={max_seq_length} \
    {'--use_lora' if use_lora else ''}
    {'--lora_rank='+str(lora_rank) if use_lora else ''}
    {'--lora_alpha='+str(lora_alpha) if use_lora else ''}
    {'--lora_dropout='+str(lora_dropout) if use_lora else ''}
    --do_train \
    --preprocessing_num_workers=16 \
    --per_device_train_batch_size={batch_size_per_gpu} \
    --gradient_accumulation_steps={gradient_acc_steps} \
    --learning_rate=2e-5 \
    --lr_scheduler_type=linear \
    --warmup_ratio=0.03 \
    --optim={optimizer} \
    --weight_decay=0. \
    --evaluation_strategy="no" \
    --logging_steps=1 \
    --save_strategy={save_strategy} \
    --save_steps={save_steps} \
    --save_total_limit=1 \
    --num_train_epochs={num_train_epochs} \
    {'--fsdp="'+fsdp+'"' if fsdp else ''}
    {'--fsdp_transformer_layer_cls_to_wrap="'+fsdp_transformer_layer_cls_to_wrap+'"' 
        if fsdp else ''}
    {'--gradient_checkpointing' if gradient_checkpointing  else ''}
    --report_to=tensorboard \
    --torch_dtype={torch_dtype} \
    --dataloader_num_workers=8 \
    {f'--{mixed_precision}=True' if mixed_precision else ''} \
    {'--overwrite_output_dir' if overwrite_output_dir else ''} \
    --output_dir="{output_dir}" \
    {'--deepspeed='+deepspeed if deepspeed else ''} \
""" 
#    --overwrite_cache

cmd = multiline_to_singleline(cmd)
if test_run:
    print()
    print(cmd)

shell_scripts = shell_scripts_template.format(
    cmd=cmd,
    log_dir=os.getcwd(),
    save_dir=output_dir)
out = submit_job(
    shell_scripts, 
    job_name=job_name, 
    nodes=nodes,
    num_cpus=num_cpus,
    cpu_mem=cpu_mem,
    num_gpus=num_gpus,
    gpu_type=gpu_type,
    test_run=test_run,
    job_duration=job_duration,
)
if not test_run:
    print(out)

Training results/baselines/huggyllama/llama-7b using 6 GPUs, 2 batch size per GPU, 2 gradient accumulation steps.

Submiting job with:
{
    "job_name": "ft:unnatural_instructions",
    "nodes": 5,
    "num_cpus": 144,
    "cpu_mem": 512,
    "num_gpus": 6,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 3
}
[{'args': 'sbatch --job-name=ft:unnatural_instructions --partition=el8 --nodes=5 --ntasks-per-node=1 --cpus-per-task=144 --mem=512GB --gres=gpu:6 --output=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/%J_1:3.out --time=6:00:00 /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/tmp3faerpqy', 'job_id': 694108}, {'args': 'sbatch --job-name=ft:unnatural_instructions --partition=el8 --nodes=5 --ntasks-per-node=1 --cpus-per-task=144 --mem=512GB --gres=gpu:6 --output=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/%J_2:3.out --time=6:00:00 --dependency=afterany:69

In [ ]:
import json

with open(error_file, 'r') as f:
    s = json.load(f)
s = s['message']
print(s['message'])
print()
print(s['extraInfo']['timestamp'])
print(s['extraInfo']['py_callstack'])

# eval



In [6]:
def get_resource_for_task(task_name):
    batch_size = 10
    if task_name == 'gsm':
        job_duration = 1 # 10min for n=200
    elif task_name == 'bbh_s=0':
        job_duration = 1
    elif task_name == 'bbh_s=3':
        job_duration = 1
        batch_size = 5 # for longer prompts.
    elif task_name == 'mmlu':
        job_duration = 1
    elif task_name == 'humaneval':
        job_duration = 1 # pass@1: 10min, pass@10: 100min
    else:
        job_duration = 1
    return batch_size, job_duration
    

In [7]:
task_name = 'mmlu'
task_name = 'gsm'
task_name = 'bbh_s=0'
task_name = 'bbh_s=3'
task_name = 'humaneval'
job_name = f'eval.{task_name}'
job_name = 'eval'

test_run = 1
test_run = bool(test_run)
use_chat_format = True
eval_final_model = True

num_cpus = 10; cpu_mem = 32 # mem usage quite small for llama7b+lora on bbh
num_cpus = 24; cpu_mem = 64

load_in_8bit = False

models = []
# models += ['t5-small', 't5-base', 't5-large', 't5-3b', 't5-11b']
# models += ['t5-11b']
# models += ['google/flan-t5-small', 'google/flan-t5-base', 'google/flan-t5-large', 'google/flan-t5-xl', 'google/flan-t5-xxl']
# models += ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
# models += [os.path.join('results/baselines', x) for x in [
#     'huggyllama/llama-7b',  # , 'mosaicml/mpt-7b'
# ]]

# models += ['results/huggyllama:llama-7b_human_mix-trainer_savebystep/']
# models = [glob.glob(os.path.join(x, 'checkpoint*')) for x in models]
# models = [x for l in models for x in l]

# models += [x if eval_final_model else get_last_checkpoint(x) for x in [
#     'results/huggyllama:llama-7b+lora(r=128,a=128)_humanmix',
#     'results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix',
#     'results/huggyllama:llama-7b_humanmix',
# ]]

datasets = [
#     'super_ni', 
#     'cot', 
    'flan_v2', 'dolly', 'oasst1',
    'gpt4_alpaca', 'stanford_alpaca', 'code_alpaca', 'baize', 'self_instruct', 
#     'sharegpt',
#     'unnatural_instructions',
#     'humanmix',
]

models = [f'results/llama-7b_{x}' for x in datasets]


task_names = [
#     'mmlu',
    'gsm',
#     'bbh_s=0',
    'humaneval',
]

options_list = itertools.product(
    task_names,
    models,
)

info = {}  
cmds = []
for task_name, model_name_or_path in options_list:
    job_name = f'eval.{task_name}'
    batch_size, job_duration = get_resource_for_task(task_name)
    
    run_id = model_name_or_path
    save_dir = f'{model_name_or_path}/eval/{task_name}'
    if use_chat_format:
        save_dir += '_chatfmt'
    
    if task_name == 'mmlu':
        cmd = f"""
        python -m eval.mmlu.run_eval \
            --data_dir data/eval/mmlu \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --ntrain 0 \
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name == 'gsm':
        cmd = f"""
        python -m eval.gsm.run_eval \
            --data_dir data/eval/gsm/ \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --max_num_examples 200 \
            --n_shot 8 \
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name.startswith('bbh'):
        cmd = f"""
        python -m eval.bbh.run_eval \
            --data_dir data/eval/bbh/ \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            {'--no_cot' if 's=0' in task_name else ''}
            {'--use_chat_format' if use_chat_format else ''}
        """
    if task_name.startswith('humaneval'):
        cmd = f"""
        python -m eval.codex_humaneval.run_eval \
            --data_file data/eval/codex_humaneval/HumanEval.jsonl.gz \
            --model_name_or_path "{model_name_or_path}" \
            --save_dir "{save_dir}" \
            --eval_batch_size {batch_size} \
            --eval_pass_at_ks 1 \
            --unbiased_sampling_size_n 1 \
            --temperature 0.1 \
            {'--use_chat_format' if use_chat_format else ''}
            {'--load_in_8bit' if load_in_8bit else ''}
        """
        
    cmd = multiline_to_singleline(cmd)
    cmds.append(cmd)
    print(cmd)
    
    # submit
    shell_scripts = shell_scripts_template.format(
        cmd=cmd,
        log_dir=os.getcwd(),
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=job_name,
        num_cpus=num_cpus,
        cpu_mem=cpu_mem,
        num_gpus=1,
        test_run=test_run,
        job_duration=job_duration,
    )
    

#     if test_run: print(out['jbsub_cmd'])
    
    

python -m eval.gsm.run_eval --data_dir data/eval/gsm/ --model_name_or_path "results/llama-7b_flan_v2" --save_dir "results/llama-7b_flan_v2/eval/gsm_chatfmt" --eval_batch_size 10 --max_num_examples 200 --n_shot 8 --use_chat_format

Submiting job with:
{
    "job_name": "eval.gsm",
    "num_cpus": 24,
    "cpu_mem": 64,
    "num_gpus": 1,
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python -m eval.gsm.run_eval --data_dir data/eval/gsm/ --model_name_or_path "results/llama-7b_dolly" --save_dir "results/llama-7b_dolly/eval/gsm_chatfmt" --eval_batch_size 10 --max_num_examples 200 --n_shot 8 --use_chat_format

Submiting job with:
{
    "job_name": "eval.gsm",
    "num_cpus": 24,
    "cpu_mem": 64,
    "num_gpus": 1,
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python -m eval.gsm.run_eval --data_dir data/eval/gsm/ --model_name_or_path "results/llama-7b_oasst1" --save_dir "results/llama-7b_oasst1/eval/gsm_chatfmt" --eval_batch_size 10 --max_num_examples 200 -

In [84]:
!cd .. && CUDA_VISIBLE_DEVICES=0 python -m eval.gsm.run_eval --data_dir data/eval/gsm/ --model_name_or_path "results/llama-7b_cot" --save_dir "results/llama-7b_cot/eval/gsm_chatfmt" --eval_batch_size 10 --max_num_examples 200 --n_shot 8 --use_chat_format


[2023-07-26 16:32:07,845] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Loading data...
Loading model and tokenizer...
Generating Completions:   0%|                           | 0/200 [00:00<?, ?it/s]/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
^C



KeyboardInterrupt



In [11]:
class EvalResults:
    
    def __init__(self, save_dir, run_name=None):
        self.save_dir = save_dir
        self.eval_dir = os.path.join(self.save_dir, 'eval')
        self.run_name = run_name if run_name else self.save_dir
        
    def get_result_df(self):

        task_names = os.listdir(self.eval_dir)
        print(task_names)

        dfs = []
        for task_name in task_names:
            task_save_dir = os.path.join(self.eval_dir, task_name)
            metrics_file = os.path.join(task_save_dir, 'metrics.json')
            if not os.path.exists(metrics_file):
                continue
            with open(metrics_file, 'r') as f:
                metrics = json.load(f)

            if 'mmlu' in task_name:
                for k, v in metrics['subcat_acc'].items():
                    metrics[k] = v
                del metrics['subcat_acc']
                for k, v in metrics['cat_acc'].items():
                    metrics[k] = v
                del metrics['cat_acc']

            columns = [f'{task_name}/{k}' for k in metrics.keys()]
            # columns = pd.MultiIndex.from_tuples(columns)
            data = list(metrics.values())

            df = pd.DataFrame([data], columns=columns)
            dfs.append(df)

        df = pd.concat(dfs, axis=1)
        mapper = {
            'mmlu/average_acc': 'MMLU/0-shot',
            'mmlu_chatfmt/average_acc': 'MMLU/0-shot_chatfmt',
            'gsm/exact_match': 'GSM/CoT',
            'gsm_chatfmt/exact_match': 'GSM/CoT_chatfmt',
            'bbh_s=0/average_exact_match': 'BBH/Direct',
            'bbh_s=0_chatfmt/average_exact_match': 'BBH/Direct_chatfmt',
            'bbh_s=3/average_exact_match': 'BBH/CoT', 
            'bbh_s=3_chatfmt/average_exact_match': 'BBH/CoT_chatfmt',
            'humaneval/pass@1': 'Codex-Eval/Pass@1',
            'humaneval_chatfmt/pass@1': 'Codex-Eval/Pass@1_chatfmt',
        }
        cols = [col for col in mapper.keys() if col in df]
        for col in cols:
            df[col] = df[col]*100
        df = df[cols]
        df = df.rename(columns=mapper)
        df.insert(0, 'Model', [self.run_name])
        return df

# get_last_checkpoint(v)
save_dirs = []
save_dirs += [
    ('llama-7b', '../results/baselines/huggyllama/llama-7b/'),
#     ('llama-7b+humanmix', '../results/huggyllama:llama-7b_humanmix'),
#     ('llama-7b+lora(r=8,a=8)', '../results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix'),
#     ('llama-7b+lora(r=128,a=128)', '../results/huggyllama:llama-7b+lora(r=128,a=128)_humanmix'),
]
datasets = [
    'super_ni', 'cot', 'flan_v2', 'dolly', 'oasst1',
    'gpt4_alpaca', 'stanford_alpaca', 'code_alpaca', 'baize', 'self_instruct', 
#     'sharegpt',
#     'unnatural_instructions',
#     'humanmix',
]

save_dirs += [(f'llama-7b+{x}', f'../results/llama-7b_{x}') for x in datasets]

dfs = []
for model_name, save_dir in save_dirs:
    r = EvalResults(save_dir, model_name)
    df = r.get_result_df()
    dfs.append(df)

pd.set_option('display.max_colwidth', None)

df = pd.concat(dfs, axis=0)
with pd.option_context('display.float_format', '{:0.1f}'.format):
    dfc = df.copy()
    cols = [x.split('_') for x in df.columns]
    cols = [x+[''] if len(x)==1 else x for x in cols]
    dfc.columns = pd.MultiIndex.from_tuples(cols)
    display(dfc)

['bbh_s=0_chatfmt', 'gsm_chatfmt', 'bbh_s=3_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'gsm', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']
['bbh_s=0_chatfmt', 'gsm_chatfmt', 'mmlu_chatfmt', 'humaneval_chatfmt']


,Model,MMLU/0-shot,GSM/CoT,BBH/Direct,Codex-Eval/Pass@1
,,chatfmt,chatfmt,chatfmt,chatfmt
0,llama-7b,32.5,11.0,33.0,5.5
0,llama-7b+super_ni,43.5,3.0,34.1,7.9
0,llama-7b+cot,37.0,28.0,34.9,10.4
0,llama-7b+flan_v2,43.2,12.0,34.4,12.8
0,llama-7b+dolly,37.2,13.0,30.6,11.6
0,llama-7b+oasst1,34.1,7.5,29.7,2.4
0,llama-7b+gpt4_alpaca,39.2,8.5,31.7,12.2
0,llama-7b+stanford_alpaca,41.7,10.5,32.9,11.0
0,llama-7b+code_alpaca,34.9,11.0,31.6,14.6


In [ ]:
{"dataset": "flan_v2", "id": "flan_v2_2", "messages": [{"role": "user", "content": "Tratatul de la Lisabona nu face inutil referire, pentru prima dată în istoria Uniunii Europene, la drepturile persoanelor care aparţin acestor minorităţi şi la valorile proprii acestora.\n\nWhich language is this?\n"}, {"role": "assistant", "content": "Romanian"}]}
{"dataset": "flan_v2", "id": "flan_v2_2", "messages": [{"role": "user", "content": "Tratatul de la Lisabona nu face inutil referire, pentru prima dat\u0103 \u00een istoria Uniunii Europene, la drepturile persoanelor care apar\u0163in acestor minorit\u0103\u0163i \u015fi la valorile proprii acestora.\n\nWhich language is this?\n"}, {"role": "assistant", "content": "Romanian"}]}


In [ ]:
with pd.option_context('display.float_format', '{:0.1f}'.format):
    display(df[['Model']+[x for x in df.columns if 'chatfmt' in x]])

In [ ]:
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(df[[x for x in df.columns if 'chatfmt' not in x]])


In [ ]:
import glob

models = []
models += ['t5-small', 't5-base', 't5-large', 't5-3b', 't5-11b']
models += ['huggyllama/llama-7b']
save_dirs = [f'../results/baselines/{x}/eval/gsm/' for x in models]

data = []
for model, save_dir in zip(models, save_dirs):
    logfile_path = glob.glob(os.path.join(save_dir, '*.out'))[0]
    out = get_run_statistics(logfile_path)
    with open(os.path.join(save_dir, 'metrics.json'), 'r') as f:
        metrics = json.load(f)
    data.append((model, out['cpu_time']/60/60, out['avg_mem'], out['max_mem'], metrics['exact_match']))
    

import pandas as pd
columns = ['name', 'cpu_time (hr)', 'avg_mem', 'max_mem', 'exact_match']
df = pd.DataFrame(data, columns=columns)
df

In [ ]:


df

